# **PRÉ-PROCESSAMENTO**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

In [20]:
df1 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_1s2022.csv", delimiter=';')
df2 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_2s2022.csv", delimiter=';')
df3 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_2s2021.csv", delimiter=';')
df4 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_1s2023.csv", delimiter=';')

In [21]:
frames = [df1, df2, df3, df4]
df = pd.concat(frames)

In [4]:
df.to_csv('zppoutput.csv', sep=';', encoding='utf-8', index = False)

In [26]:
df.head()

,ID,Order Number,Posting date,SKU,Quantity,Milestone,Entered by,Cost ctr
0,1820587547445640OO9417L94170359MARIANAAN,1820587547,03/01/2022,0OO9417L94170359,60,PRA,MARIANAAN,132-45565
1,1820871421445640AX4121S82948759CINTHIABDAR,1820871421,03/01/2022,0AX4121S82948759,37,LIN,CINTHIABDAR,132-45565
2,1821294046445640JK2007BJ16354GICELES,1821294046,03/01/2022,0JK2007BJ16354,69,EMB,GICELES,132-45565
3,1821294046445640JK2007BJ16354ALIDIA,1821294046,03/01/2022,0JK2007BJ16354,69,LIN,ALIDIA,132-45565
4,1821404980445640RB3663L004/7160MARIANAAN,1821404980,03/01/2022,0RB3663L004/7160,319,PRA,MARIANAAN,132-45565


## **Transformando o dataframe para exibir as variáveis de interesse**

In [22]:
df2 = pd.DataFrame.copy(df)

In [23]:
# Removendo colunas que não serão utilizadas
df2.drop('ID', axis=1, inplace=True)
df2.drop('Milestone', axis=1, inplace=True)
df2.drop('SKU', axis=1, inplace=True)
df2.drop('Order Number', axis=1, inplace=True)
df2.head()

,Posting date,Quantity,Entered by,Cost ctr
0,03/01/2022,60,MARIANAAN,132-45565
1,03/01/2022,37,CINTHIABDAR,132-45565
2,03/01/2022,69,GICELES,132-45565
3,03/01/2022,69,ALIDIA,132-45565
4,03/01/2022,319,MARIANAAN,132-45565


In [24]:
# Fazendo um agrupamento dos dados em função das variáveis 'Cost ctr' e 'Posting date' para se obter as variáveis 'QTY' e 'Employees_QTY'
df3 = pd.DataFrame({'QTY': df2.groupby(by=['Cost ctr', 'Posting date'])['Quantity'].sum(),
                    'Employees_QTY': df2.groupby(['Cost ctr', 'Posting date']).size()}).reset_index()
df3

,Cost ctr,Posting date,QTY,Employees_QTY
0,132-07265,01/02/2022,1160,11
1,132-07265,01/02/2023,3330,13
2,132-07265,01/03/2023,1650,12
3,132-07265,01/04/2022,1320,18
4,132-07265,01/04/2023,1490,6
...,...,...,...,...
3608,132-45565,31/07/2021,43740,349
3609,132-45565,31/08/2021,76291,603
3610,132-45565,31/08/2022,66944,808
3611,132-45565,31/10/2022,58424,528


## **Transformando as variáveis categóricas nominais em variáveis categóricas ordinais**

In [25]:
df3['Cost ctr'].replace({'132-07265':0, '132-07565': 1, '132-10365':2, '132-31265': 3, '132-31865': 4, '132-45565': 5}, inplace=True)
df3['Posting date'] = np.arange(len(df3.index))
df3.head()

,Cost ctr,Posting date,QTY,Employees_QTY
0,0,0,1160,11
1,0,1,3330,13
2,0,2,1650,12
3,0,3,1320,18
4,0,4,1490,6


In [29]:
df3.rename(columns={"Cost ctr": "Cost_ctr", "Posting date": "Posting_date"}, inplace=True)
df3

,Cost_ctr,Posting_date,QTY,Employees_QTY
0,0,0,1160,11
1,0,1,3330,13
2,0,2,1650,12
3,0,3,1320,18
4,0,4,1490,6
...,...,...,...,...
3608,5,3608,43740,349
3609,5,3609,76291,603
3610,5,3610,66944,808
3611,5,3611,58424,528


In [28]:
df3.dtypes

Cost ctr         int64
Posting date     int32
QTY              int64
Employees_QTY    int64
dtype: object

## **LEGENDA**

Cost_ctr = centro de custo (setor da linha de produção)

Posting_date = data de produção

QTY = quantidade de peças produzidas

Employees QTY = quantidade de funcionários

## **ATRIBUTOS PREVISORES E ALVO**

In [10]:
# Definindo atributos que serão previsores e alvo de previsão
previsores = df3.iloc[:, 0:3].values
previsores

array([[    0,     0,  1160],
       [    0,     1,  3330],
       [    0,     2,  1650],
       ...,
       [    5,  3610, 66944],
       [    5,  3611, 58424],
       [    5,  3612,   300]], dtype=int64)

In [17]:
previsores.shape

(3613, 3)

In [11]:
alvo = df3.iloc[:, 3].values
alvo

array([ 11,  13,  12, ..., 808, 528,   1], dtype=int64)

In [20]:
alvo.shape

(3613,)

## **Análise das escalas dos atributos (Escalonamento)**

In [30]:
df3.describe()

,Cost_ctr,Posting_date,QTY,Employees_QTY
count,3613.000000,3613.000000,3613.000000,3613.000000
mean,2.603930,1806.000000,27463.464157,205.261279
std,1.670888,1043.127589,25948.141979,226.514060
min,0.000000,0.000000,50.000000,1.000000
25%,1.000000,903.000000,5273.000000,36.000000
50%,3.000000,1806.000000,17269.000000,107.000000
75%,4.000000,2709.000000,46587.000000,305.000000
max,5.000000,3612.000000,103292.000000,1039.000000


Padronização (utiliza a média e o desvio padrão como referência).

Normalização (utiliza os valores máximo e mínimo como referência).

In [13]:
previsores_esc = StandardScaler().fit_transform(previsores)
previsores_esc

array([[-1.55862659, -1.73157148, -1.01383386],
       [-1.55862659, -1.73061269, -0.93019395],
       [-1.55862659, -1.7296539 , -0.99494743],
       ...,
       [ 1.43420816,  1.7296539 ,  1.52172748],
       [ 1.43420816,  1.73061269,  1.19333482],
       [ 1.43420816,  1.73157148, -1.04698148]])

## **RESUMO PRÉ-PROCESSAMENTO**

alvo = variável que se pretende atingir (quantidade de funcionários por setor).

previsores = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas manualmente, sem escalonar.

previsores_esc = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas, escalonada.

## **BASE DE TREINO E TESTE**

Parâmetros train_test_split:   
- arrays: nomes dos atributos previsores e alvo.   
- test_size: tamanho em porcentagem dos dados de teste. default é none.   
- train_size: tamanho em porcentagem dos dados de treinamento.default é none.  
- random_state: nomeação de um estado aleatório.   
- shuffle: embaralhamento dos dados aleatórios. Associado com o random_state ocorre o mesmo embaralhamento sempre. Default é True.  
- stratify: Possibilidade de dividir os dados de forma estratificada. Default é None (nesse caso é mantido a proporção, isto é, se tem 30% de zeros e 70% de 1 no dataframe, na separação em treinamento e teste se manterá essa proporção).

In [14]:
x_treino, x_teste, y_treino, y_teste = train_test_split(previsores, alvo, test_size = 0.3, random_state = 0)

In [15]:
x_treino.shape

(2529, 3)

In [16]:
x_teste.shape

(1084, 3)

In [17]:
y_treino.shape

(2529,)

In [18]:
y_teste.shape

(1084,)

## **Salvando as variáveis (atributos)**

In [31]:
df3.to_csv('zppoutput_processed.csv', sep=';', encoding='utf-8', index = False)